# **Download**



In [ ]:
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 3.3MB 10.8MB/s 
     |████████████████████████████████| 2.2MB 10.8MB/s 
     |████████████████████████████████| 870kB 21.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=9e79d6c9781481daadbae9d291151661aa1061073d562ce779be6975a96e7404
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


# **Model**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import SequenceEnqueuer
import pickle as pkl
import seaborn as sns
import os
import matplotlib.pyplot as plt
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
from tqdm import tqdm_notebook as tqdm

In [ ]:
!unzip Dat.zip

Streaming output truncated to the last 5000 lines.
  inflating: dat/267trn_img.jpg      
  inflating: dat/268trn_img.jpg      
  inflating: dat/269trn_img.jpg      
  inflating: dat/26trn_img.jpg       
  inflating: dat/270trn_img.jpg      
  inflating: dat/271trn_img.jpg      
  inflating: dat/272trn_img.jpg      
  inflating: dat/273trn_img.jpg      
  inflating: dat/274trn_img.jpg      
  inflating: dat/275trn_img.jpg      
  inflating: dat/276trn_img.jpg      
  inflating: dat/277trn_img.jpg      
  inflating: dat/279trn_img.jpg      
  inflating: dat/27trn_img.jpg       
  inflating: dat/280trn_img.jpg      
  inflating: dat/281trn_img.jpg      
  inflating: dat/282trn_img.jpg      
  inflating: dat/283trn_img.jpg      
  inflating: dat/284trn_img.jpg      
  inflating: dat/285trn_img.jpg      
  inflating: dat/286trn_img.jpg      
  inflating: dat/287trn_img.jpg      
  inflating: dat/288trn_img.jpg      
  inflating: dat/289trn_img.jpg      
  inflating: dat/28trn_img.jpg       

In [ ]:
!ls

dat  dat.csv  Dat.zip  drive  ids.pkl  img.csv	msk.pkl  sample_data


In [ ]:
!nvidia-smi

Thu Apr  8 04:59:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tqdm import tqdm as tq

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img
from tensorflow.keras.preprocessing.image import load_img as load_img
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
tq.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
!ls

dat  dat.csv  Dat.zip  drive  ids.pkl  img.csv	msk.pkl  sample_data


In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, df, x_col_tup, y_tup,target_size=(125,125) ,batch_size=32, shuffle=True):
        self.batch_size = batch_size
        self.df = df
        self.target_size=target_size
        self.indices = self.df.index.tolist()
        self.shuffle = shuffle
        self.x_col_tup = x_col_tup
        self.y_tup = y_tup
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, ix):
      try:
        ix = self.index[ix * self.batch_size:(ix + 1) * self.batch_size]
      except:
        ix = self.index[(ix * self.batch_size)[0]:((ix + 1) * self.batch_size)[0]]
      j=[i for i in ix if i<len(self.indices)]
      batch = [self.indices[k] for k in j]
      X,y=self.__get_data(batch)
      return X, y

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def __get_data(self, batch):
        batch_df=self.df.iloc[batch]
        
        X = []
        for i in self.x_col_tup:
            x_col,x_type=i
            x_dat=batch_df[x_col].values
            if (x_type=="img"):
                x_dat=np.array(list(map(lambda x:img_to_array(load_img(x, color_mode="rgb", target_size=self.target_size, interpolation="nearest"))/255, x_dat)))
                X.append(x_dat)
            elif (x_type=="cate"):
                x_dat=to_categorical(x_dat)
                X.append(x_dat)
            elif (x_type=="scalar"):
                X.append(np.asarray(x_dat,dtype=np.float32))
            elif (x_type=="check"):
                X.append(np.asarray([np.asarray(i,dtype=np.float32) for i in x_dat],dtype=np.float32))
            elif (x_type=="checkbool"):
                X.append(np.asarray([np.asarray(i,dtype=np.int32) for i in x_dat],dtype=np.int32))
            else:
                X.append(x_dat)
        y=[]
        y_col, y_type = self.y_tup
        y_dat=batch_df[y_col].values
        if (y_type=="img"):
            y_dat=np.array(list(map(lambda x:img_to_array(load_img(x, color_mode="rgb", target_size=self.target_size, interpolation="nearest"))/255, y_dat)))
            y.append(y_dat)
        elif (y_type=="cate"):
            y_dat=to_categorical(y_dat)
            y.append(y_dat)
        elif (x_type=="scalar"):
            y.append(np.asarray(y_dat,dtype=np.float32))
        else:
            y.append(y_dat)
        

        return X, y

    def __iter__(self):
        for item in (self[i] for i in range(len(self))):
          yield item

In [ ]:
### LOAD BERT TOKENIZERS ###
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=False)

In [ ]:
l=pd.read_csv("img.csv")

In [ ]:
l["img"].values[0]

'dat/0trn_img.jpg'

In [ ]:
ids=pkl.load(open("ids.pkl","rb"))
ids=pad_sequences(ids,50,padding="post")
msk=pkl.load(open("msk.pkl","rb"))
msk=pad_sequences(msk,50,padding="post")
img=l["img"].values

In [ ]:
df=pd.DataFrame.from_dict({"wrd_ids":list(ids),"wrd_msk":list(msk),"img":list(img)})

In [ ]:
def imgTest(i):
  try:
    if os.stat(i).st_size>=500 :
      return True
    else:
      return False
  except:
    return False

In [ ]:
df=df.loc[df["img"].progress_apply(imgTest)]

100%|██████████| 25648/25648 [00:00<00:00, 158580.10it/s]


In [ ]:
df=df.reset_index()

In [ ]:
len(df)

25558

In [ ]:
df["label"]=np.ones((len(df),1))

In [ ]:

# clip model weights to a given hypercube
class ClipConstraint(tf.keras.constraints.Constraint):
	# set clip value when initialized
  def __init__(self, clip_value):
    self.clip_value = clip_value
 
	# clip model weights to hypercube
  def __call__(self, weights):
    return tf.keras.backend.clip(weights, -self.clip_value, self.clip_value)
 
	# get the config
  def get_config(self):
    return {'clip_value': self.clip_value}

In [33]:
class CGAN():
    def __init__(self,latent_dim,img_shape,max_seq):
      self.out_shape=img_shape
      self.max_seq=max_seq
      self.latent_dim = latent_dim
      optimizer = tf.keras.optimizers.RMSprop(0.00005)
      optimizer_g=tf.keras.optimizers.Adam(0.00005)
            # Build and compile the discriminator
      self.discriminator = self.build_discriminator()
      self.discriminator.compile(loss=self.wasserstein_loss,
          optimizer=optimizer)

      # Build the generator
      self.generator = self.build_generator()

      # The generator takes noise and the target label as input
      # and generates the corresponding digit of that label
      noise = tf.keras.layers.Input(shape=(self.latent_dim,))
      ids = tf.keras.layers.Input(shape=(50,))
      msk = tf.keras.layers.Input(shape=(50,))
      img = self.generator([noise, ids,msk])

      # For the combined model we will only train the generator
      self.discriminator.trainable = False

      # The discriminator takes generated image as input and determines validity
      # and the label of that image
      valid = self.discriminator([img, ids,msk])

      # The combined model  (stacked generator and discriminator)
      # Trains generator to fool discriminator
      self.combined = tf.keras.Model([noise, ids,msk], valid)
      self.combined.compile(loss=self.wasserstein_loss,optimizer=optimizer_g)
    
    def wasserstein_loss(self, y_true, y_pred):
        return tf.keras.backend.mean(y_true * y_pred)

    def build_generator(self):
      input_noise=tf.keras.layers.Input(self.latent_dim)
      noise_den=tf.keras.layers.Dense(128)(input_noise)
      den_chai=tf.keras.layers.Dense(128,activation="gelu")(noise_den)
      den_chai2=tf.keras.layers.Dense(128,activation="gelu")(den_chai)
      input_ids=tf.keras.layers.Input(self.max_seq)
      input_msk=tf.keras.layers.Input(self.max_seq,dtype=np.bool)
      wrd_emb=tf.keras.layers.Embedding(tokenizer.get_vocab_size()+1,64)(input_ids)
      q=tf.keras.layers.Dense(128)(wrd_emb)
      k=tf.keras.layers.Dense(128)(wrd_emb)
      v=tf.keras.layers.Dense(128)(wrd_emb)
      att=tf.keras.layers.Attention()([q,k,v],[input_msk,input_msk])
      den_en=tf.keras.layers.Dense(128,activation="gelu")(att[:,0,:])
      den_en2=tf.keras.layers.Dense(128,activation="gelu")(den_en)
      concat=tf.keras.layers.Concatenate()([den_chai2,den_en2])
      den_0=tf.keras.layers.Dense(128,activation="gelu")(concat)
      den_1=tf.keras.layers.Dense(512,activation="gelu")(den_0)
      den_2=tf.keras.layers.Dense(1024,activation="gelu")(den_1)
      den_res=tf.keras.layers.Dense(np.prod(self.out_shape),activation="sigmoid")(den_2)
      res=tf.keras.layers.Reshape(self.out_shape,name="kaI")(den_res)
      model=tf.keras.models.Model(inputs=[input_noise,input_ids,input_msk],outputs=res)
      model.summary()

      return tf.keras.models.Model(inputs=[input_noise,input_ids,input_msk],outputs=res)

    def build_discriminator(self):
      input_img=tf.keras.layers.Input(self.out_shape)
      input_ids=tf.keras.layers.Input(self.max_seq)
      input_msk=tf.keras.layers.Input(self.max_seq,dtype=np.bool)
      wrd_emb=tf.keras.layers.Embedding(tokenizer.get_vocab_size()+1,64)(input_ids)

      q=tf.keras.layers.Dense(128)(wrd_emb)
      k=tf.keras.layers.Dense(128)(wrd_emb)
      v=tf.keras.layers.Dense(128)(wrd_emb)
      att=tf.keras.layers.Attention()([q,k,v],[input_msk,input_msk])
      const = ClipConstraint(0.02)
      den_en=tf.keras.layers.Dense(128,activation="gelu", kernel_constraint=const)(att[:,0,:])
      den_en2=tf.keras.layers.Dense(128,activation="gelu", kernel_constraint=const)(den_en)
      
      conv_1=tf.keras.layers.Conv2D(32,5,activation="gelu", kernel_constraint=const)(input_img)
      mxpl_1=tf.keras.layers.MaxPool2D((5,5))(conv_1)
      conv_2=tf.keras.layers.Conv2D(32,5,activation="gelu", kernel_constraint=const)(mxpl_1)
      mxpl_2=tf.keras.layers.MaxPool2D((5,5))(conv_2)
      flat=tf.keras.layers.Flatten()(mxpl_2)
      den_1=tf.keras.layers.Dense(2048,activation="gelu", kernel_constraint=const)(flat)
      den_2=tf.keras.layers.Dense(1024,activation="gelu", kernel_constraint=const)(den_1)
      concat=tf.keras.layers.Concatenate()([den_2,den_en])
      
      den_3=tf.keras.layers.Dense(512,activation="gelu", kernel_constraint=const)(concat)
      lst_den=tf.keras.layers.Dense(256,activation="gelu", kernel_constraint=const)(den_3)
      res=tf.keras.layers.Dense(1)(den_3)
      model=tf.keras.models.Model(inputs=[input_img,input_ids,input_msk],outputs=res)
      model.summary()

      return tf.keras.models.Model(inputs=[input_img,input_ids,input_msk],outputs=res)

    
    def train(self,df, epochs, batch_size=32, sample_interval=50,save_interval=200,save_path=""):
      gen=DataGenerator(df,[("img","img"),("wrd_ids","check"),("wrd_msk","checkbool")],("label","check"),batch_size=batch_size)
      valid = -np.ones((batch_size, 1))
      fake = np.ones((batch_size, 1))
      for epoch in range(epochs):
        for i in tqdm(gen.__iter__(),total=gen.__len__()):
          noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
          gen_imgs = self.generator.predict([noise,i[0][1],i[0][2]])
          d_loss_real = self.discriminator.train_on_batch(list(i[0]), valid)
          d_loss_fake = self.discriminator.train_on_batch([gen_imgs,i[0][1],i[0][2]], fake)
          d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
          for l in range(2):
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch([noise,i[0][1],i[0][2]], valid)
        print('>epoch:'+str(epoch+1)+', cri-real='+str(d_loss_real)+', cri-fake='+str(d_loss_fake)+' g='+str(g_loss))
        if epoch % sample_interval == 0:
            self.sample_images(epoch)
        if epoch % save_interval == 0:
            self.save_model(save_path)


    def sample_images(self, epoch):
        r, c = 2, 5
        gen=DataGenerator(df,[("wrd_ids","check"),("wrd_msk","checkbool")],("label","checkbool"),batch_size=r*c)
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        i=np.random.randint(100, size=1)
        label=gen.__getitem__(i)
        gen_imgs = self.generator.predict([noise, label[0][0],label[0][1]])
        fig, axs = plt.subplots(r, c,figsize=(10, 15))
        cnt = 0
        lst=np.asarray([i.astype(np.int32) for i in label[0][0][cnt]],dtype=np.int32)
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,:], cmap='gray')
                axs[i,j].set_title("Sentence: "+ tokenizer.decode(lst))
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

    def save_model(self,path=""):
      self.discriminator.save_weights(path+"discriminatorTrained.h5")
      self.generator.save_weights(path+"generatorTrained.h5")
      
    def load_model(self,discrim_file,gen_file):
      self.discriminator.load_weights(discrim_file)
      self.generator.load_weights(gen_file)
      optimizer = tf.keras.optimizers.Adam(0.00005)
      optimizer_g=tf.keras.optimizers.Adam(0.00005)
      noise = tf.keras.layers.Input(shape=(self.latent_dim,))
      ids = tf.keras.layers.Input(shape=(50,))
      msk = tf.keras.layers.Input(shape=(50,))
      img = self.generator([noise, ids,msk])

      # For the combined model we will only train the generator
      self.discriminator.trainable = False

      # The discriminator takes generated image as input and determines validity
      # and the label of that image
      valid = self.discriminator([img, ids,msk])

      # The combined model  (stacked generator and discriminator)
      # Trains generator to fool discriminator
      self.combined = tf.keras.Model([noise, ids,msk], valid)
      self.combined.compile(loss=self.wasserstein_loss,optimizer=optimizer_g)


  
    def generate_image(self,filename,text="mtf"):
      if text=="mtf":
        r, c = 2, 5
        gen=DataGenerator(df,[("wrd_ids","check"),("wrd_msk","checkbool")],("label","checkbool"),batch_size=r*c)
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        i=np.random.randint(20000, size=1)
        label=gen.__getitem__(i)
        gen_imgs = self.generator.predict([noise, label[0][0],label[0][1]])
        fig, axs = plt.subplots(r, c,figsize=(10, 15))
        cnt = 0
        lst=np.asarray([i.astype(np.int32) for i in label[0][0][cnt]],dtype=np.int32)
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,:], cmap='gray')
                axs[i,j].set_title("Sentence: "+ tokenizer.decode(lst))
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images_gen/"+filename+".png")
        plt.close()
      else:
        gen=DataGenerator(df,[("wrd_ids","check"),("wrd_msk","checkbool")],("label","checkbool"),batch_size=1)
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        print (noise)
        x_ids_tr=tokenizer.encode(text).ids
        ids=pad_sequences([x_ids_tr],self.max_seq,padding="post")
        x_msk_tr=tokenizer.encode(text).attention_mask
        msk=pad_sequences([x_msk_tr],self.max_seq,padding="post")
        gen_img = self.generator.predict([noise,ids,msk])
        save_img("images_gen/"+filename+'.jpg', gen_img[0])





In [36]:
cgan=CGAN(225,(125,125,3),50)

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 125, 125, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 121, 121, 32) 2432        input_34[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 24, 24, 32)   0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
input_35 (InputLayer)           [(None, 50)]         0                                            
___________________________________________________________________________________________

In [35]:
cgan.load_model("/content/drive/Shareddrives/Chula Drive/Datasets/MemeGen/discriminatorTrained.h5","/content/drive/Shareddrives/Chula Drive/Datasets/MemeGen/generatorTrained.h5")

KeyboardInterrupt: ignored

In [ ]:
cgan.train(df,1500,batch_size=1024,save_path="/content/drive/Shareddrives/Chula Drive/Datasets/MemeGen/")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:100: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



>epoch:1, cri-real=-0.04088827595114708, cri-fake=0.005074469838291407 g=0.00318915955722332



>epoch:2, cri-real=-0.45621585845947266, cri-fake=0.6804144382476807 g=-0.5994059443473816



>epoch:3, cri-real=-0.030730297788977623, cri-fake=0.12948444485664368 g=-0.09261009097099304



>epoch:4, cri-real=0.3157872259616852, cri-fake=-1.7205373048782349 g=2.058490037918091


...


>epoch:17, cri-real=5.62443208694458, cri-fake=-1.340741753578186 g=1.3560638427734375



>epoch:18, cri-real=0.1863357424736023, cri-fake=-0.028270935639739037 g=0.03834664449095726



>epoch:19, cri-real=-4.575503349304199, cri-fake=1.050701379776001 g=-0.7767186760902405



>epoch:20, cri-real=-7.320866584777832, cri-fake=13.349238395690918 g=-12.272987365722656



>epoch:21, cri-real=-3.9123330116271973, cri-fake=6.713900089263916 g=-6.167782783508301


In [ ]:
cgan.generate_image("Omg2","I am chicken I am egg")

[[ 0.65364117 -1.21622035  0.44092515 -1.13925175  0.40408761  0.20250298
  -0.05109422 -0.14370844 -0.36257757  0.2353277  -0.79603467  2.12065831
   0.70173766 -0.27379114 -0.60575548  0.71786628  0.1898531   1.28624764
   1.75234476  0.33285659 -1.46168113 -0.47502623  1.54218407 -1.02941536
   1.11282632  0.29611539  0.28429357 -0.30726225 -0.76301293  0.3592863
   0.87730233  1.57308627  0.82933983  0.20185429 -0.01492623 -0.16569784
   0.39220251  0.98896068 -0.18540532 -0.322975   -1.81584606 -0.20972346
   0.5968466   0.18276428 -0.613146    0.3951372  -0.34830973  0.52211936
   0.11706613  0.91385597  1.58923214  0.65146551  0.03936436 -0.34141836
  -0.7676271   0.02456941 -0.72653554 -1.21570137 -1.42221683  1.70834333
   0.16457085 -0.50303161  0.08277058  1.003591    0.92153937  1.13897826
  -0.83418742  2.40437805  0.20843839 -3.20022003 -0.87811028  0.61208057
  -0.7641568  -1.85164967 -0.84648602 -1.00952002  0.0274042  -0.49621449
   2.08672811  1.42055953 -0.9253515  -

In [ ]:
path="/content/drive/Shareddrives/Chula Drive/Datasets/MemeGen/"

In [ ]:
from google.colab import files
files.download("discriminatorTrained.h5")
files.download("generatorTrained.h5")

In [ ]:
!nvidia-smi

In [ ]:
gen=DataGenerator(df,[("wrd_ids",None),("wrd_msk",None)],("label",None),batch_size=16)

In [ ]:
max(df.index.to_list())

In [ ]:
l=[17156, 421, 8951, 13979, 7254, 11465, 16980, 1860, 20470, 13100, 18745, 18398, 20827, 14834, 2844, 25647]

In [ ]:
[i for i in l if i<10000]

In [ ]:
len(df)